In [1]:
import cv2
import numpy as np

from mtcnn.mtcnn import MTCNN

import tkinter as tk

In [2]:
# -----------------------------------------------------------------------------
# Función para detectar y extraer el rostro en la segunda imagen
# -----------------------------------------------------------------------------
def extraer_rostro(image, face_cascade):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    # Verifica si se detectó algún rostro; si no, devuelve una imagen en negro
    if len(faces) == 0:
        print("No se detectó rostro en la imagen de la otra persona.")
        return np.zeros_like(image)
    
    # Extrae y devuelve el primer rostro detectado en la imagen
    (x, y, w, h) = faces[0]
    return image[y:y+h, x:x+w]

# -----------------------------------------------------------------------------
# Función para el modo de cambio de caras
# -----------------------------------------------------------------------------
def modo_cambio_de_caras(frame, other_face, faces, min_face_size):
    
    for (x, y, w, h) in faces:
        # Verifica si el rostro detectado cumple con el tamaño mínimo
        if w >= min_face_size:
            resized_other_face = cv2.resize(other_face, (w, h))

            # Verifica si la imagen del rostro tiene un canal alfa para usar la transparencia
            if resized_other_face.shape[2] == 4:
                alpha_mask = resized_other_face[:, :, 3] / 255.0
                alpha_inv = 1 - alpha_mask
                
                # Aplica la superposición con transparencia sobre el rostro detectado
                for c in range(3):  # BGR canales
                    frame[y:y+h, x:x+w, c] = (alpha_mask * resized_other_face[:, :, c] + alpha_inv * frame[y:y+h, x:x+w, c])
            else:
                # Si no tiene canal alfa, simplemente superpone la imagen sin transparencia
                frame[y:y+h, x:x+w] = resized_other_face
    return frame


In [13]:
# -----------------------------------------------------------------------------
# Función para el modo del sombrero
# -----------------------------------------------------------------------------
def modo_sombrero(frame, sombrero, faces, min_face_size):
    
    for (x, y, w, h) in faces:
        # Verificar si el rostro detectado cumple con el tamaño mínimo
        if w >= min_face_size:

            # Redimensionar el sombrero para que sea un poco más ancho que el rostro
            new_width = int(w * 1.2)
            new_height = int(new_width * sombrero.shape[0] / sombrero.shape[1])
            sombrero_resized = cv2.resize(sombrero, (new_width, new_height))
            
            # Calcular la posición para colocar el sombrero encima de la cabeza
            y_offset = y - sombrero_resized.shape[0]
            y_offset = max(0, y_offset)  # Evitar salir del límite superior de la imagen

            x_offset = x - (sombrero_resized.shape[1] - w) // 2
            x_offset = max(0, x_offset)  # Evitar salir del límite izquierdo de la imagen

            # Superponer el sombrero en la imagen utilizando el canal alfa para transparencia
            for i in range(sombrero_resized.shape[0]):
                for j in range(sombrero_resized.shape[1]):
                    if sombrero_resized[i, j, 3] != 0:  # Comprobar que el píxel es visible
                        if 0 <= y_offset + i < frame.shape[0] and 0 <= x_offset + j < frame.shape[1]:
                            frame[y_offset + i, x_offset + j] = sombrero_resized[i, j, :3]
    return frame

In [4]:
# -----------------------------------------------------------------------------
# Función para el modo de gafas
# -----------------------------------------------------------------------------
def modo_gafas(frame, gafas, faces, eyepair_cascade, min_eye_size):
    
    # Convertir el fotograma a escala de grises para detección
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    for (x, y, w, h) in faces:
        # Detectar solo si el rostro es suficientemente grande
        
        if w >= min_eye_size:
            # Detectar la región de los ojos dentro del rostro
            roi_gray = gray[y:y+h, x:x+w]
            eyepairs = eyepair_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)

            # Verificar si se detecta un par de ojos
            if len(eyepairs) > 0:
                (ex, ey, ew, eh) = eyepairs[0]

                # Calcular el tamaño de las gafas para ajustarlas a la región de los ojos
                g_width = int(1.6 * ew)  # Ancho ajustado de las gafas
                g_height = int(g_width * gafas.shape[0] / gafas.shape[1])
                gafas_resized = cv2.resize(gafas, (g_width, g_height))

                # Calcular la posición donde se colocarán las gafas en el fotograma
                y_offset = y + ey - int(0.4 * g_height)
                x_offset = x + ex - int(0.2 * g_width)

                # Superponer las gafas sobre el fotograma, utilizando el canal alfa para la transparencia
                for i in range(gafas_resized.shape[0]):
                    for j in range(gafas_resized.shape[1]):
                        if gafas_resized[i, j, 3] != 0:  # Verificar transparencia en el canal alfa
                            if 0 <= y_offset + i < frame.shape[0] and 0 <= x_offset + j < frame.shape[1]:
                                frame[y_offset + i, x_offset + j] = gafas_resized[i, j, :3]
    return frame


In [5]:
# -----------------------------------------------------------------------------
# Función para superponer una imagen con transparencia
# -----------------------------------------------------------------------------
def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
    
    # Obtener las coordenadas de la posición de superposición
    x, y = pos
    overlay_h, overlay_w = img_overlay.shape[:2]

    # Extraer la región de interés (ROI) en la imagen base
    roi = img[y:y+overlay_h, x:x+overlay_w]

    # Superponer la imagen usando la máscara de transparencia alfa
    for c in range(3):  # Aplicar para cada canal de color (BGR)
        roi[:, :, c] = (alpha_mask * img_overlay[:, :, c] + (1 - alpha_mask) * roi[:, :, c])

    # Asignar la región procesada de nuevo a la imagen original
    img[y:y+overlay_h, x:x+overlay_w] = roi


# -----------------------------------------------------------------------------
# Función para el modo de payaso
# -----------------------------------------------------------------------------
def modo_payaso(frame, face_cascade, nose_cascade, peluca, nariz_roja, mejilla_img):
    
    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Superponer la peluca, ajustando el tamaño para cubrir toda la parte superior del rostro
        peluca_resized = cv2.resize(peluca, (int(w * 1.5), int(h * 1)))
        peluca_x = x - int(w * 0.25)
        peluca_y = y - int(h * 0.55)

        # Verificar que la peluca se puede superponer sin salir de los límites de la imagen
        if peluca_y >= 0 and peluca_x >= 0 and peluca_y + peluca_resized.shape[0] <= frame.shape[0] and peluca_x + peluca_resized.shape[1] <= frame.shape[1]:
            overlay_image_alpha(frame, peluca_resized, (peluca_x, peluca_y), peluca_resized[:, :, 3] / 255.0)

        # Detectar y superponer la nariz roja, ajustándola a un tamaño más grande y elevándola ligeramente
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        noses = nose_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
        
        if len(noses) > 0:
            (nx, ny, nw, nh) = noses[0]
            nariz_resized = cv2.resize(nariz_roja, (int(nw * 2.6), int(nh * 2.6)))
            nariz_x = nx - int(nw * 0.75)
            nariz_y = ny - int(nh * 0.85)

            # Verificar que la nariz se puede superponer sin problemas
            if nariz_y >= 0 and nariz_x >= 0 and nariz_y + nariz_resized.shape[0] <= roi_color.shape[0] and nariz_x + nariz_resized.shape[1] <= roi_color.shape[1]:
                overlay_image_alpha(roi_color, nariz_resized, (nariz_x, nariz_y), nariz_resized[:, :, 3] / 255.0)

        # Colocar las mejillas a ambos lados de la nariz, ajustando el tamaño y posición
        mejilla_size = (int(w * 0.3), int(h * 0.3))
        mejilla_izq = cv2.resize(mejilla_img, mejilla_size)
        mejilla_der = cv2.resize(mejilla_img, mejilla_size)

        # Verificar que las mejillas se pueden superponer sin problemas
        if int(h * 0.55) + mejilla_size[1] < h:
            if int(w * 0.1) + mejilla_size[0] <= w:
                overlay_image_alpha(roi_color, mejilla_izq, (int(w * 0.1), int(h * 0.55)), mejilla_izq[:, :, 3] / 255.0)
            if int(w * 0.6) + mejilla_size[0] <= w:
                overlay_image_alpha(roi_color, mejilla_der, (int(w * 0.6), int(h * 0.55)), mejilla_der[:, :, 3] / 255.0)

    return frame

In [15]:
# -----------------------------------------------------------------------------
# Configuración de clasificadores y carga de imágenes para filtros
# -----------------------------------------------------------------------------
face_cascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
eyepair_cascade = cv2.CascadeClassifier('Cascades/haarcascade_mcs_eyepair_big.xml')
nose_cascade = cv2.CascadeClassifier('Cascades/haarcascade_mcs_nose.xml')

sombrero = cv2.imread('hat.png', cv2.IMREAD_UNCHANGED)
other_face = extraer_rostro(cv2.imread('pngimg.com - barack_obama_PNG39.png', cv2.IMREAD_UNCHANGED), face_cascade)
gafas = cv2.imread('pngtree-wayfarer-sunglasses-with-black-frames-png-image_4487246.png', cv2.IMREAD_UNCHANGED)

peluca = cv2.imread('peluca.png', cv2.IMREAD_UNCHANGED)
nariz_roja = cv2.imread('nariz roja.png', cv2.IMREAD_UNCHANGED)
mejilla_img = cv2.imread('mejilla.png', cv2.IMREAD_UNCHANGED)

# -----------------------------------------------------------------------------
# Configuración de la ventana de selección de modos
# -----------------------------------------------------------------------------
mode = 1  # Modo inicial
running = True  # Control del bucle de la cámara

ventana = tk.Tk()
ventana.title("Selector de Modo")
ventana.geometry("200x300")

def cambiar_modo(nuevo_modo):
    global mode
    mode = nuevo_modo

def cerrar_camara():
    global running
    running = False

# Botones de selección de modo y cierre de cámara
botones_info = [("Sombrero de Vaquero", "red", 1), ("Obama", "blue", 2), ("Gafas", "green", 3), ("Payaso", "purple", 4)]
for texto, color, modo in botones_info:
    tk.Button(ventana, text=texto, command=lambda m=modo: cambiar_modo(m), bg=color, fg="white").pack(fill="both", expand=True, padx=10, pady=5)

tk.Button(ventana, text="Cerrar Cámara", command=cerrar_camara, bg="black", fg="white").pack(fill="both", expand=True, padx=10, pady=5)

# -----------------------------------------------------------------------------
# Bucle principal de captura de video y aplicación de filtros
# -----------------------------------------------------------------------------
cap = cv2.VideoCapture(0)

while running:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Aplicación de filtro según el modo seleccionado
    if mode == 1:
        frame = modo_sombrero(frame, sombrero, faces, 150)
    elif mode == 2:
        frame = modo_cambio_de_caras(frame, other_face, faces, 125)
    elif mode == 3:
        frame = modo_gafas(frame, gafas, faces, eyepair_cascade, 10)
    elif mode == 4:
        frame = modo_payaso(frame, face_cascade, nose_cascade, peluca, nariz_roja, mejilla_img)

    # Mostrar el fotograma procesado
    cv2.imshow('Video con filtro', cv2.resize(frame, (600, 400)))
    ventana.update()

# Liberar la cámara y cerrar ventanas
cap.release()
cv2.destroyAllWindows()
ventana.destroy()